### Necessary packages

In [1]:
# Common imports
from datetime import datetime
import numpy as np 
import pandas as pd
pd.options.mode.chained_assignment = None # avoid error 
pd.options.display.float_format = '{:.5f}'.format # No scientific annotation when print dataframe
from functools import reduce

# ADA and Modeling imports
from scipy.stats import stats
import tensorflow as tf
from tensorflow import keras

# To plot pretty figures
import seaborn as sns
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


# Build function
from HelperFunctions import *
from Implementation import *

# To generate an stable output across runs
rnd_seed = 42
rnd_gen = np.random.default_rng(rnd_seed)


### Importing Data

In [2]:
LakeConfig, pars,HydroSource,StratSource,HypoSource = ModelConfig(2)
StratData, dataseq = DataPreparation_Mendota(StratSource)
Inflow =  GetInFlow(StratData.index,HydroSource)
HypsoME,NewHypso,obsCDEpi,obsCDHypo,PeakDates,PeakEpi,PeakHypo=  GetLakeData(HypoSource)

pars = ( 0.0141929, 22.8315205,  1.0651408,  1.1718688)

Pepi,Phyp,Psed= MechanisticModels(LakeConfig,pars,HydroSource,StratSource,HypoSource,StratData,Inflow,HypsoME,NewHypso,obsCDEpi,obsCDHypo,PeakDates,PeakEpi,PeakHypo)


In [3]:
def Volume_calc(NewHypso,LakeConfig,StratData,dataseq):
    #### Lake Config Values ####
    MeanDepth = (LakeConfig.MeanDepth).item() #m
    # nex line is to generate a variable with the same value (25) but with a len of obsTherm to not get error in calculation
    zm = (LakeConfig.zm).item()#max depth in m
    LakeArea = (LakeConfig.LakeArea).item() #m2
    MeanDepth = (LakeConfig.MeanDepth).item() #m
    LakeVolume = (LakeArea* MeanDepth)
    
    if HypsoME.Hypso_cum.values is None:
        
        obsTherm = pd.DataFrame(round(StratData['Therm'], 1))
        # [0] -> to just grab the first value of zm and not get and error for doing math with variables with diff shape
        A = ((3*((2*(MeanDepth/zm))-1))*(((zm-obsTherm.Therm)/zm)**2)) - ((2*((3*(MeanDepth/zm))-2))*((zm-obsTherm.Therm)/zm))
        V = (((zm-obsTherm.Therm)/zm)+(((3*(MeanDepth/zm))-2)*(((zm-obsTherm.Therm)/zm)**2))-((2*(MeanDepth/zm))-1)*(((zm-obsTherm.Therm)/zm)**3))/((MeanDepth/zm))
        V = 1-V
    else:
        
        obsTherm = pd.DataFrame(round(StratData['Therm'], 1))
        # Reset index to have the original index of each data frame to then extract the index in which both match
        NewHypso.reset_index(inplace = True)
        NewHypso['depth'] = round(NewHypso['depth'], 1)

        obsTherm.reset_index(inplace = True) # x2 reset in StratData since the 1st takeout sampledate from index and
        obsTherm.reset_index(inplace = True) # 2nd reset to pass the original index as a column to then match

        # Mask to find  the index position in which the Thermocline depth match with depth of hypso -> index_x
        mask =  pd.merge(obsTherm,NewHypso, left_on='Therm', right_on='depth', how = 'left')['index_y']
        V = NewHypso.Hypso_cum.iloc[mask].values
        
    # Calculate epi and hypo volume vectors, which should always add up to lake volume
    HypV = (LakeVolume * (1-V))# hypolimnetic volume time series
    EpiV = LakeVolume-HypV
    
    df = pd.DataFrame({'HypV':HypV, 'EpiV':EpiV})
    df.index = dataseq.index
    return(df)
    


In [4]:
volume = (Volume_calc(NewHypso,LakeConfig,StratData,dataseq))
obsCDEpi.rename(columns ={'totpuf_sloh':'ObsPepi'},inplace = True)
obsCDEpi.set_index('sampledate',inplace = True)
obsCDHypo.rename(columns ={'totpuf_sloh':'ObsPhyp'}, inplace = True)
obsCDHypo.set_index('sampledate',inplace = True)
Inflow.rename(columns = {'P.g.day':'PLoad','Discharge.m3.d':'Discharge'},inplace = True)

MM_P = (pd.DataFrame({'sampledate':dataseq.index,'Pepi':Pepi/volume.EpiV, 'Phyp':Phyp/volume.HypV})).set_index('sampledate')
MM_P.rename(columns = {'Pepi':'MM_Pepi','Phyp':'MM_Phyp'},inplace = True)

In [5]:
# compile the list of dataframes you want to merge
dfs = [volume, obsCDEpi, obsCDHypo,StratData,Inflow,MM_P]

df_merged =reduce(lambda a, b: pd.merge(a, b, on='sampledate', how='outer'), dfs)

df_Pepi = (df_merged[df_merged['ObsPepi'].notnull()]).drop(['ObsPhyp','MM_Phyp'], axis =1)
df_Phyp  = df_merged[df_merged['ObsPhyp'].notnull()].drop(['ObsPepi','MM_Pepi'], axis =1)
#df[['EpiTemp','HypoTemp','Strat','Therm','EpiV','HypV','Discharge','PLoad','ObsPepi','ObsPhyp']]
df_Pepi= df_Pepi[['EpiTemp','HypoTemp','Strat','Therm','EpiV','HypV','Discharge','PLoad','ObsPepi','MM_Pepi']]
df_Phyp= df_Phyp[['EpiTemp','HypoTemp','Strat','Therm','EpiV','HypV','Discharge','PLoad','ObsPhyp','MM_Phyp']]

In [6]:
df_Pepi

,EpiTemp,HypoTemp,Strat,Therm,EpiV,HypV,Discharge,PLoad,ObsPepi,MM_Pepi
sampledate,,,,,,,,,,
1995-05-09,8.56403,8.56398,False,1.00000,56777379.32032,447462223.87968,1290796.73280,542743.32825,0.08600,0.08600
1995-05-23,12.35971,12.12482,False,1.00000,56777379.32032,447462223.87968,914623.92000,378740.21721,0.08100,0.08858
1995-06-06,16.94056,13.42717,True,10.05876,321861181.11859,182378422.08141,965757.94560,417196.11612,0.07400,0.08536
1995-06-21,24.30285,13.42861,True,9.74908,311887321.76730,192352281.43270,507135.94560,197016.38079,0.03700,0.06068
1995-07-06,23.22701,13.43011,True,9.41257,304338854.90739,199900748.29261,273875.12640,110060.51558,0.04200,0.03541
...,...,...,...,...,...,...,...,...,...,...
2015-08-03,24.57979,9.49851,True,6.89510,238712070.55091,265527532.64909,169462.62720,52681.40688,0.02800,0.02746
2015-08-18,24.62611,9.50271,True,7.18038,246845455.35053,257394147.84947,268401.25440,78759.07449,0.02200,0.02137
2015-08-31,21.64207,9.50959,True,7.55281,257575674.10662,246663929.09338,292992.59520,100750.15499,0.02600,0.02117


In [8]:
df_Pepi.to_csv('Pepi_mendota.csv')

### Machine learning model

Recurrent Neural Network

In [7]:
# Let's clear out the backend and set our random seeds
# Consistency makes things easier for labs!
keras.backend.clear_session()
tf.random.set_seed(rnd_seed)
np.random.seed(rnd_seed)